# Fusion lumiplanscrap

In [36]:
import pandas as pd
start_index = 0
end_index = 17 
directory = 'Lumiplan_scrap_0.csv'
combined_df = pd.DataFrame()
for index in range(start_index, end_index + 1):
    if index !=3:
        df = pd.read_csv(f'Lumiplan_scrap_{index}.csv')
        combined_df = pd.concat([combined_df, df], ignore_index=True)


Remise en forme avec ast

In [37]:
import ast
combined_df['Pistes'] = combined_df['Pistes'].apply(ast.literal_eval)
combined_df['Remontées'] = combined_df['Remontées'].apply(ast.literal_eval)
combined_df['detail'] = combined_df['detail'].apply(ast.literal_eval)
del combined_df["Unnamed: 0"]

In [38]:
# Liste des stations de ski spécifiques
stations_2_flocons_vert = ['les-arcs', 'saint-gervais', 'valberg', 'lans-en-vercors', 'chamonix', 'Chatel']

stations_1_etoile = ['grandbornand', 'morzine', 'valloire', 'les-angles', 'tignes', 'saint-francois-longchamp', 'megeve', 'Val-d-isere', 'les-contamines-montjoie', 'crest-voland', 'les-saisies', 'la-rosiere']

# Ajoutez une nouvelle colonne et attribuez la valeur correspondante
combined_df['nombre_flocon_vert'] = 0  # Initialisez la nouvelle colonne à 0

# Attribuez 2 pour les stations avec 2 flocons verts
combined_df.loc[combined_df['station_name'].isin(stations_2_flocons_vert), 'nombre_flocon_vert'] = 2

# Attribuez 1 pour les stations avec 1 étoile
combined_df.loc[combined_df['station_name'].isin(stations_1_etoile), 'nombre_flocon_vert'] = 1


print(combined_df[['station_name', 'nombre_flocon_vert']])

                        station_name  nombre_flocon_vert
0                          abondance                   0
1                    albiez-montrond                   0
2                            ancelle                   0
3     champsaur-valgaudemar-nordique                   0
4                   areches-beaufort                   0
...                              ...                 ...
1970                        valloire                   1
1971                      valmeinier                   0
1972                            vars                   0
1973                         vaujany                   0
1974                 villard-de-lans                   0

[1975 rows x 2 columns]


Compte du nombre de station ouverte ou fermé

In [39]:

def count_ski_status(pistes):
    ski_counts = {'Open': 0, 'Closed': 0, 'Green_closed': 0, 'Blue_closed': 0, 'Red_closed': 0, 'Black_closed': 0,
                  'Closed_Unknown': 0, 'Open_Unknown': 0, 'Green_open': 0, 'Blue_open': 0, 'Red_open': 0, 'Black_open': 0}

    for item in pistes:
        _, status, color, activity = item

        if activity == 'SKI':
            if status == 0:
                ski_counts['Closed'] += 1
                if isinstance(color, int):
                    if color == 0:
                        ski_counts['Green_closed'] += 1
                    elif color == 1:
                        ski_counts['Blue_closed'] += 1
                    elif color == 2:
                        ski_counts['Red_closed'] += 1
                    elif color == 3:
                        ski_counts['Black_closed'] += 1
                    else:
                        ski_counts['Closed_Unknown'] += 1
            elif status == 1:
                ski_counts['Open'] += 1
                if isinstance(color, int):
                    if color == 0:
                        ski_counts['Green_open'] += 1
                    elif color == 1:
                        ski_counts['Blue_open'] += 1
                    elif color == 2:
                        ski_counts['Red_open'] += 1
                    elif color == 3:
                        ski_counts['Black_open'] += 1
                    else:
                        ski_counts['Open_Unknown'] += 1

    return ski_counts


combined_df['Ski_Counts'] = combined_df['Pistes'].apply(count_ski_status)

print(combined_df)

                        station_name  \
0                          abondance   
1                    albiez-montrond   
2                            ancelle   
3     champsaur-valgaudemar-nordique   
4                   areches-beaufort   
...                              ...   
1970                        valloire   
1971                      valmeinier   
1972                            vars   
1973                         vaujany   
1974                 villard-de-lans   

                                                    url  \
0     https://bulletinv3.lumiplan.pro/bulletin.php?s...   
1     https://bulletinv3.lumiplan.pro/bulletin.php?s...   
2     https://bulletin.lumiplan.pro/bulletin.php?sta...   
3     https://bulletin.lumiplan.pro/bulletin.php?sta...   
4     https://bulletinv3.lumiplan.pro/bulletin.php?s...   
...                                                 ...   
1970  https://bulletinv3.lumiplan.pro/bulletin.php?s...   
1971  https://bulletinv3.lumiplan.pro/bulletin.

In [40]:
import plotly.express as px
def count_remontee_status(remontee):
    remontee_counts = {'Open_remontee': 0, 'Closed_remontee': 0}

    i = 0
    for item in remontee:
        if len(item) ==3:
            _, status, activity = item
            if activity == 'remontée': 
                    if status == 0:
                        remontee_counts['Closed_remontee'] += 1
                    elif status == 1:
                        remontee_counts['Open_remontee'] += 1
    return remontee_counts

combined_df['remontee_Counts'] = combined_df['Remontées'].apply(count_remontee_status)


In [46]:

daily_counts = combined_df.groupby('scrap_date')['Ski_Counts'].agg(
    lambda x: {
        'Open': sum(y['Open'] for y in x),
        'Closed': sum(y['Closed'] for y in x),
        'Green_closed': sum(y['Green_closed'] for y in x),
        'Green_open': sum(y['Green_open'] for y in x),
        'Blue_closed': sum(y['Blue_closed'] for y in x),
        'Blue_open': sum(y['Blue_open'] for y in x),
        'Red_closed': sum(y['Red_closed'] for y in x),
        'Red_open': sum(y['Red_open'] for y in x),
        'Black_closed': sum(y['Black_closed'] for y in x),
        'Black_open': sum(y['Black_open'] for y in x),
    }
).reset_index()


daily_counts[['Open', 'Closed', 'Green_closed', 'Green_open', 'Blue_closed', 'Blue_open', 'Red_closed', 'Red_open', 'Black_closed', 'Black_open']] = pd.json_normalize(daily_counts['Ski_Counts'])

fig = px.line(daily_counts, x='scrap_date', y=['Open', 'Closed', 'Green_closed', 'Green_open', 'Blue_closed', 'Blue_open', 'Red_closed', 'Red_open', 'Black_closed', 'Black_open'],
              title='Evolution of Open and Closed Ski Slopes by Color')
fig.update_layout(xaxis_title='Date', yaxis_title='Count', legend_title='Status')
fig.show()

In [51]:

daily_counts = combined_df.groupby('scrap_date')['remontee_Counts'].agg(
    lambda x: {
        'Open_remontee': sum(y['Open_remontee'] for y in x),
        'Closed_remontee': sum(y['Closed_remontee'] for y in x),
    }
).reset_index()


daily_counts[['Open_remontee', 'Closed_remontee']] = pd.json_normalize(daily_counts['remontee_Counts'])

fig = px.line(daily_counts, x='scrap_date', y=['Open_remontee', 'Closed_remontee'])
fig.update_layout(xaxis_title='Date', yaxis_title='Count', legend_title='Status')
fig.show()

In [44]:
unique_dates = combined_df['scrap_date'].unique()
print(unique_dates)


['2023-12-13' '2023-12-17' '2023-12-18' '2023-12-26' '2023-12-27'
 '2023-12-30' '2024-01-05' '2024-01-07' '2024-01-08' '2024-01-09'
 '2024-01-10' '2024-01-11' '2024-01-12' '2024-01-13' '2024-01-14'
 '2024-01-15' '2024-01-16']


In [57]:
combined_df["Ski_Counts"][0]

{'Open': 0,
 'Closed': 14,
 'Green_closed': 4,
 'Blue_closed': 8,
 'Red_closed': 1,
 'Black_closed': 1,
 'Closed_Unknown': 0,
 'Open_Unknown': 0,
 'Green_open': 0,
 'Blue_open': 0,
 'Red_open': 0,
 'Black_open': 0}

In [50]:
combined_df = combined_df[combined_df['scrap_date'] != '2024-01-11']


In [52]:
combined_df.to_csv("Lumiplan_formatted_flocon.csv")